# Building an AI Book Research Assistant

This notebook provides step-by-step instructions for replicating what Andrew did in the second demo. Please follow along, and feel free to play with different variations too! At the end, there are optional instructions for building a more advanced book research assistant than Andrew showed. 

*Useful Resources*:

- **AISuite GitHub Repo**: [https://github.com/andrewyng/aisuite](https://github.com/andrewyng/aisuite)
- **Open Library API**:
  - [Open Library](https://openlibrary.org/developers/api) offers free and public Web APIs for accessing book and author catalog data (no API key is required)
  - See `openlibrary_api_docs.md` in this lesson's directory for more information

## Step 1: Use Open Library API's Documentation

- Open Jupyter chat by clicking on the chat bubble icon on the left sidebar of Jupyter Lab
- In the Chat interface, create a new chat by clicking `+Chat` (choose any name for the file)
- In the chat window, attach the Markdown file `openlibrary_api_docs.md` using the `@` symbol:
  - Type `@`, then select `file` from the autocomplete menu
  - You'll see a list of available files in the lesson's directory, choose `openlibrary_api_docs.md` 
- Use a prompt like this:
   > Write a code cell that uses Open Library's search endpoint to look up books based on a user's query and returns a response listing identified books, with these fields for each book: key, title, author_name, first_publish_year, isbn, cover_id, edition_count, language, subject. Use a variable called limit, set to 5, to specify the number of returned results. Print the results. Use the Open Library API documentation in the attached file.
- Transfer the generated code to the cell below and run it

In [1]:
import requests

# User query and result limit
query = "python programming"  # Example query - you can change this
limit = 5

# Define the fields we want to return
fields = "key,title,author_name,first_publish_year,isbn,cover_i,edition_count,language,subject"

# Make request to Open Library Search API
response = requests.get(
    'https://openlibrary.org/search.json',
    params={
        'q': query,
        'fields': fields,
        'limit': limit
    }
)

# Parse JSON response
data = response.json()

# Print results
print(f"Search query: '{query}'")
print(f"Number of results found: {data.get('num_found', 0)}")
print(f"Showing {len(data.get('docs', []))} results:\n")

for i, book in enumerate(data.get('docs', []), 1):
    print(f"Book {i}:")
    print(f"  Key: {book.get('key', 'N/A')}")
    print(f"  Title: {book.get('title', 'N/A')}")
    print(f"  Author(s): {', '.join(book.get('author_name', ['N/A']))}")
    print(f"  First Published: {book.get('first_publish_year', 'N/A')}")
    print(f"  ISBN(s): {', '.join(book.get('isbn', ['N/A'])[:3])}")  # Show first 3 ISBNs
    print(f"  Cover ID: {book.get('cover_i', 'N/A')}")
    print(f"  Edition Count: {book.get('edition_count', 'N/A')}")
    print(f"  Language(s): {', '.join(book.get('language', ['N/A']))}")
    print(f"  Subject(s): {', '.join(book.get('subject', ['N/A'])[:5])}")  # Show first 5 subjects
    print()

Search query: 'python programming'
Number of results found: 2485
Showing 5 results:

Book 1:
  Key: /works/OL30906747W
  Title: Core Python Programming
  Author(s): R. Nageswara Rao
  First Published: 2016
  ISBN(s): 9351199428, 9789390457151, 9789351199427
  Cover ID: 13192647
  Edition Count: 5
  Language(s): hin, eng
  Subject(s): N/A

Book 2:
  Key: /works/OL12409294W
  Title: Core Python programming
  Author(s): Wesley Chun
  First Published: 2006
  ISBN(s): 0137061595, 0132269937, 9780132269933
  Cover ID: 7972455
  Edition Count: 2
  Language(s): eng
  Subject(s): Python (Computer program language)

Book 3:
  Key: /works/OL19547345W
  Title: Black Hat Python
  Author(s): Justin Seitz, Tim Arnold
  First Published: 2014
  ISBN(s): 1718501129, 8328312506, 9781718501126
  Cover ID: 8513526
  Edition Count: 3
  Language(s): pol, eng
  Subject(s): Python (Computer program language), Computer security, Python (język programowania), Sieci komputerowe, Dostęp

Book 4:
  Key: /works/OL26

## Step 2: Wrap the API Call in a Tool Function

- In the chat interface, use a prompt like this:
  > Wrap the code above in a function `search_books`, with good docstring.
  > 
  > - Input arguments: `query` and `limit` (with default value of 10)
  > - Output: JSON string with:
  >   - `num_found`: total results
  >   - `books`: list of dicts with title, authors, first_publish_year, key, edition_count

In [2]:
import json
import requests


def search_books(query: str, limit: int = 10) -> str:
    """
    Search for books using the Open Library API.
    
    This function queries the Open Library search endpoint to find books
    matching the provided search query and returns structured information
    about the results.
    
    Args:
        query (str): Search query string (e.g., "python programming", 
                     "science fiction", "Jane Austen")
        limit (int, optional): Maximum number of books to return. 
                               Defaults to 10.
    
    Returns:
        str: JSON string containing:
            - num_found (int): Total number of results found in the database
            - books (list): List of dictionaries, each containing:
                - title (str): Book title
                - authors (list): List of author names
                - first_publish_year (int): Year of first publication
                - key (str): Unique Open Library work identifier
                - edition_count (int): Number of editions available
    
    Example:
        >>> results = search_books("python programming", limit=5)
        >>> data = json.loads(results)
        >>> print(f"Found {data['num_found']} books")
        >>> for book in data['books']:
        ...     print(f"- {book['title']} by {', '.join(book['authors'])}")
    """
    # Define the fields we want to return
    fields = "key,title,author_name,first_publish_year,edition_count"
    
    # Make request to Open Library Search API
    response = requests.get(
        'https://openlibrary.org/search.json',
        params={
            'q': query,
            'fields': fields,
            'limit': limit
        }
    )
    
    # Parse JSON response
    data = response.json()
    
    # Structure the results
    result = {
        'num_found': data.get('num_found', 0),
        'books': []
    }
    
    # Extract relevant information for each book
    for book in data.get('docs', []):
        book_info = {
            'title': book.get('title', 'N/A'),
            'authors': book.get('author_name', []),
            'first_publish_year': book.get('first_publish_year', 'N/A'),
            'key': book.get('key', 'N/A'),
            'edition_count': book.get('edition_count', 'N/A')
        }
        result['books'].append(book_info)
    
    # Return as JSON string
    return json.dumps(result, indent=2)

- Test the function by uncommenting the code below:

In [3]:
results = search_books("science fiction")
print(results)

{
  "num_found": 99770,
  "books": [
    {
      "title": "The Science Fiction Hall of Fame -- Volume One",
      "authors": [
        "Robert Silverberg",
        "Robert A. Heinlein",
        "Arthur C. Clarke",
        "Isaac Asimov",
        "Ray Bradbury"
      ],
      "first_publish_year": 1970,
      "key": "/works/OL1960472W",
      "edition_count": 31
    },
    {
      "title": "Foundation",
      "authors": [
        "Isaac Asimov"
      ],
      "first_publish_year": 1951,
      "key": "/works/OL46125W",
      "edition_count": 97
    },
    {
      "title": "Stranger in a Strange Land",
      "authors": [
        "Robert A. Heinlein"
      ],
      "first_publish_year": 1961,
      "key": "/works/OL59688W",
      "edition_count": 80
    },
    {
      "title": "The Last Man",
      "authors": [
        "Mary Shelley"
      ],
      "first_publish_year": 1826,
      "key": "/works/OL450124W",
      "edition_count": 383
    },
    {
      "title": "The Time Machine",
      "

## Step 3: Build a Chat Handler with Automatic Function Calling

- Drag and drop the raw cell below into the chat window and use a prompt like this:
  > Use this aisuite example to create a simple chat handler function `chat_with_tools`:
  > - input: `user_message`
  > - output: `response.choices[0].message.content`

In [4]:
from dotenv import load_dotenv
load_dotenv()

import aisuite as ai

def chat_with_tools(user_message):
    """
    Simple chat handler function using aisuite with automatic tool execution.
    
    Args:
        user_message (str): The user's message/query
        
    Returns:
        str: The response content from the AI
    """
    client = ai.Client()
    
    messages = [{
        "role": "user",
        "content": user_message
    }]
    
    # Automatic tool execution with max_turns
    response = client.chat.completions.create(
        model="openai:gpt-4o-mini",
        messages=messages,
        tools=[search_books],
        max_turns=3  # Maximum number of back-and-forth tool calls
    )
    
    return response.choices[0].message.content

## Step 4: Test the Chat Handler

- Run the following code cell to test `chat_with_tools`
- Feel free to search for different types of books
- Optional: If your generated function `search_books` does not contain printing statements, you can update the function as Andrew showed in the video:
  - prompt used: modify the search_books function to add a printing statement showing when it is called with basic status messages.

In [5]:
print("="*60)
print("EXAMPLE: Science Function book search")
print("="*60)

user_query = "Search for books about science fiction"
print(f"\n👤 User: {user_query}")

response = chat_with_tools(user_query)

print(f"\n🤖 Assistant: {response}")

EXAMPLE: Science Function book search

👤 User: Search for books about science fiction

🤖 Assistant: Here are some notable books about science fiction:

1. **[The Science Fiction Hall of Fame -- Volume One](https://openlibrary.org/works/OL1960472W)**
   - **Authors:** Robert Silverberg, Robert A. Heinlein, Arthur C. Clarke, Isaac Asimov, Ray Bradbury
   - **First Published:** 1970
   - **Edition Count:** 31

2. **[Foundation](https://openlibrary.org/works/OL46125W)**
   - **Author:** Isaac Asimov
   - **First Published:** 1951
   - **Edition Count:** 97

3. **[Stranger in a Strange Land](https://openlibrary.org/works/OL59688W)**
   - **Author:** Robert A. Heinlein
   - **First Published:** 1961
   - **Edition Count:** 80

4. **[The Last Man](https://openlibrary.org/works/OL450124W)**
   - **Author:** Mary Shelley
   - **First Published:** 1826
   - **Edition Count:** 383

5. **[The Time Machine](https://openlibrary.org/works/OL52267W)**
   - **Author:** H. G. Wells
   - **First Publishe

## Step 5: Look up Details on Books (optional) 

You can repeat steps 1-3 to define a second tool for the book research assistant, so it can handle more complex book queries.
For example, you can use the `works` endpoint to retrieve more detailed information about a particular book given its work id. For more details, check `openlibrary_api_docs.md`.

Here are steps with suggested prompts:

1. In the chat interface, attach the file containing the API documentation of Open Library (`openlibrary_api_docs.md`) and use a prompt like this:
   > Write a code cell that uses Open Library's works endpoint to retrieve information about a book, based on the work's key that has this format: "/works/{work_id}". The result should include: title, description, subjects (limit to 10), first_sentence, covers (limit to 3). Print the returned results. Use the documentation for Open Library's API provided above.
2. In the chat interface, create the function tool using a prompt like this:
   > Wrap the code above in a function `get_book_details`:
   > - Function argument: `work_key` in this format: `/works/{work_id}`
   > - Output: JSON string with: title, description, subjects (limit to 10), first_sentence, covers (limit to 3)
   >  - Write good docstring
3. Update the `chat_with_tools` function to include the second tool as follows:
   
   ``` python
   response = client.chat.completions.create(
    model="openai:gpt-4.1-mini",
    messages=messages,
    tools=[search_books, get_book_details],
    max_turns=3  # Maximum number of back-and-forth tool calls
   )
   ```
   
4. Try this query:
   ```python
   user_query = "Search for books with 'Linear Algebra' in the title and tell me about the one with the most editions"
   ```

In [ ]:
import requests

# Example work key from the search results above
work_key = "/works/OL46125W"  # Foundation by Isaac Asimov

# Make request to Open Library Works API
response = requests.get(f'https://openlibrary.org{work_key}.json')

# Parse JSON response
data = response.json()

# Extract and structure the relevant fields
result = {
    'title': data.get('title', 'N/A'),
    'description': data.get('description', 'N/A'),
    'subjects': data.get('subjects', [])[:10],  # Limit to 10
    'first_sentence': data.get('first_sentence', 'N/A'),
    'covers': data.get('covers', [])[:3]  # Limit to 3
}

# Handle description if it's a dict with 'value' key
if isinstance(result['description'], dict):
    result['description'] = result['description'].get('value', 'N/A')

# Handle first_sentence if it's a dict with 'value' key
if isinstance(result['first_sentence'], dict):
    result['first_sentence'] = result['first_sentence'].get('value', 'N/A')

# Print the results
print(f"Work Key: {work_key}")
print(f"\nTitle: {result['title']}")
print(f"\nDescription: {result['description']}")
print("\nSubjects (up to 10):")
for subject in result['subjects']:
    print(f"  - {subject}")
print(f"\nFirst Sentence: {result['first_sentence']}")
print(f"\nCover IDs (up to 3): {result['covers']}")

Work Key: /works/OL46125W

Title: Foundation

Description: One of the great masterworks of science fiction, the Foundation novels of Isaac Asimov are unsurpassed for their unique blend of nonstop action, daring ideas, and extensive world-building. 

The story of our future begins with the history of Foundation and its greatest psychohistorian: Hari Seldon.  For twelve thousand years the Galactic Empire has ruled supreme. Now it is dying.  Only Hari Seldon, creator of the revolutionary science of psychohistory, can see into the future--a dark age of ignorance, barbarism, and warfare that will last thirty thousand years. To preserve knowledge and save mankind, Seldon gathers the best minds in the Empire--both scientists and scholars--and brings them to a bleak planet at the edge of the Galaxy to serve as a beacon of hope for future generations. He calls his sanctuary the Foundation.

But soon the fledgling Foundation finds itself at the mercy of corrupt warlords rising in the wake of the

In [7]:
import requests


def get_book_details(work_key: str) -> str:
    """
    Retrieve detailed information about a book using the Open Library Works API.
    
    This function fetches comprehensive details about a specific book using its
    unique Open Library work identifier and returns structured information
    including title, description, subjects, first sentence, and cover IDs.
    
    Args:
        work_key (str): The Open Library work identifier in the format 
                       "/works/{work_id}" (e.g., "/works/OL46125W")
    
    Returns:
        str: JSON string containing:
            - title (str): The book's title
            - description (str): Full description of the book
            - subjects (list): List of subject categories (limited to 10)
            - first_sentence (str): The book's opening sentence
            - covers (list): List of cover image IDs (limited to 3)
    
    Example:
        >>> details = get_book_details("/works/OL46125W")
        >>> data = json.loads(details)
        >>> print(f"Title: {data['title']}")
        >>> print(f"Description: {data['description'][:100]}...")
        >>> print(f"Subjects: {', '.join(data['subjects'][:5])}")
    
    Note:
        The function handles cases where description or first_sentence might
        be stored as a dictionary with a 'value' key in the API response.
    """
    # Make request to Open Library Works API
    response = requests.get(f'https://openlibrary.org{work_key}.json')
    
    # Parse JSON response
    data = response.json()
    
    # Extract and structure the relevant fields
    result = {
        'title': data.get('title', 'N/A'),
        'description': data.get('description', 'N/A'),
        'subjects': data.get('subjects', [])[:10],  # Limit to 10
        'first_sentence': data.get('first_sentence', 'N/A'),
        'covers': data.get('covers', [])[:3]  # Limit to 3
    }
    
    # Handle description if it's a dict with 'value' key
    if isinstance(result['description'], dict):
        result['description'] = result['description'].get('value', 'N/A')
    
    # Handle first_sentence if it's a dict with 'value' key
    if isinstance(result['first_sentence'], dict):
        result['first_sentence'] = result['first_sentence'].get('value', 'N/A')
    
    # Return as JSON string
    return json.dumps(result, indent=2)

In [ ]:
from dotenv import load_dotenv
load_dotenv()

import aisuite as ai

def chat_with_tools(user_message):
    """
    Simple chat handler function using aisuite with automatic tool execution.
    
    This function creates a chat interface that can automatically use both
    search_books and get_book_details tools to answer user queries about books.
    
    Args:
        user_message (str): The user's message/query
        
    Returns:
        str: The response content from the AI
    """
    client = ai.Client()
    
    messages = [{
        "role": "user",
        "content": user_message
    }]
    
    # Automatic tool execution with max_turns
    response = client.chat.completions.create(
        model="openai:gpt-4o-mini",
        messages=messages,
        tools=[search_books, get_book_details],
        max_turns=3  # Maximum number of back-and-forth tool calls
    )
    
    return response.choices[0].message.content

In [8]:
user_query = "Search for books with 'Linear Algebra' in the title and tell me about the one with the most editions"

print(f"\n👤 User: {user_query}")

response = chat_with_tools(user_query)

print(f"\n🤖 Assistant: {response}")


👤 User: Search for books with 'Linear Algebra' in the title and tell me about the one with the most editions

🤖 Assistant: The book with the most editions related to "Linear Algebra" is **"Elementary Linear Algebra"** by **Howard Anton** and **Chris Rorres**. Here are the details:

- **Title**: Elementary Linear Algebra
- **Authors**: Howard Anton, Chris Rorres
- **First Published Year**: 1973
- **Number of Editions**: 37
- **Open Library Key**: [Link to the book](https://openlibrary.org/works/OL485862W)

This book has a significant number of editions, indicating its popularity and continued relevance in the field of linear algebra.
